In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])

        # Print the column names in the Excel file
        print("Column Names in the Excel file:")
        print(df.columns)

        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Check if 'Particulars' column is found
        if excel_particulars_column not in df.columns:
            print(f"Error: '{excel_particulars_column}' column not found in the Excel file.")
            return

        # Print the Excel data before matching
        print("\nExcel Data:")
        print(df)

        # Set to keep track of processed qcodes
        processed_qcodes = set()

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])

            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)

                # Fetch all rows
                results = cursor.fetchall()
                print(results)

                # Process the results as needed
                for result in results:
                    id, cont = result
                    print(f"Product ID: {id}, Count: {cont}")

    except Exception as e:
        print(f"Error: {e}")

# ... (rest of the code remains unchanged)
import time

def scan_qr_code(db_config):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 2 seconds between scans

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Display the frame with QR codes highlighted
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    print(f"Product found: {product_result[0]}")
                    selected_product = product_result[0]

                    # Call the function to read and insert matched data to pp_out table
                    read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product)

                    # Wait for 2 seconds before the next scan
                    time.sleep(delay_seconds)
                else:
                    print("Product not found.")

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__== "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }

    # Call the QR code scanning function
    scan_qr_code(db_config)

In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])

        # Print the column names in the Excel file
        print("Column Names in the Excel file:")
        print(df.columns)

        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Check if 'Particulars' column is found
        if excel_particulars_column not in df.columns:
            print(f"Error: '{excel_particulars_column}' column not found in the Excel file.")
            return

        # Print the Excel data before matching
        print("\nExcel Data:")
        print(df)

        # Set to keep track of processed qcodes
        processed_qcodes = set()

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])

            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                pcs = row['PCS']

                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)

                # Fetch all rows
                results = cursor.fetchall()
                print(results)

                # Process the results as needed
                for result in results:
                    id, cont = result
                    print(f"Product ID: {id}, Excel_count:{pcs}, Count: {cont}")

    except Exception as e:
        print(f"Error: {e}")

# ... (rest of the code remains unchanged)
import time

def scan_qr_code(db_config):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 2 seconds between scans

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Display the frame with QR codes highlighted
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    print(f"Product found: {product_result[0]}")
                    selected_product = product_result[0]

                    # Call the function to read and insert matched data to pp_out table
                    read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product)

                    # Wait for 2 seconds before the next scan
                    time.sleep(delay_seconds)
                else:
                    print("Product not found.")

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__== "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }

    # Call the QR code scanning functionq
    scan_qr_code(db_config)

In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product, results_data):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])

        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])

            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                pcs = row['PCS']

                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)

                # Fetch all rows
                db_results = cursor.fetchall()

                # Process the results as needed
                for db_result in db_results:
                    id, cont = db_result
                    results_data.append([id, cont, pcs])


    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close cursor and connection
        cursor.close()
        conn.close()

# ... (rest of the code remains unchanged)
import time

def scan_qr_code(db_config):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 2 seconds between scans

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Display the frame with QR codes highlighted
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    print(f"Product found: {product_result[0]}")
                    selected_product = product_result[0]

                    # Initialize results data array
                    results_data = []

                    # Call the function to read and insert matched data to pp_out table
                    read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product, results_data)

                    # Print the results
                    for result_row in results_data:
                        print(f"Product ID: {result_row[0]}, Count: {result_row[1]}, Excel_count: {result_row[2]}")

                    # Wait for 2 seconds before the next scan
                    time.sleep(delay_seconds)
                else:
                    print("Product not found.")

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__== "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }

    # Call the QR code scanning function
    scan_qr_code(db_config)


In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product, results_data):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])

        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])

            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                pcs = row['PCS']

                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)

                # Fetch all rows
                db_results = cursor.fetchall()

                # Process the results as needed
                for db_result in db_results:
                    id, cont = db_result
                    results_data.append([id, cont, pcs])
                    print(results_data)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close cursor and connection
        cursor.close()
        conn.close()

# ... (rest of the code remains unchanged)
import time

def scan_qr_code(db_config):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 2 seconds between scans

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Display the frame with QR codes highlighted
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    print(f"Product found: {product_result[0]}")
                    selected_product = product_result[0]

                    # Initialize results data array
                    results_data = []

                    # Call the function to read and insert matched data to pp_out table
                    read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product, results_data)

                    # Print the results
                    for result_row in results_data:
                        print(f"Product ID: {result_row[0]}, Count: {result_row[1]}, Excel_count: {result_row[2]}")

                    # Wait for 2 seconds before the next scan
                    time.sleep(delay_seconds)
                else:
                    print("Product not found.")

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__== "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }

    # Call the QR code scanning function
    scan_qr_code(db_config)

In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector
import time

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product, results_data):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])

        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])

            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                pcs = row['PCS']

                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)

                # Fetch all rows
                db_results = cursor.fetchall()

                # Process the results as needed
                for db_result in db_results:
                    id, cont = db_result
                    results_data.append([id, cont, pcs])


    except Exception as e:
        print(f"Error: {e}")
        print(result_data)
    finally:
        # Close cursor and connection
        cursor.close()
        conn.close()

def scan_qr_code(db_config, excel_file_path):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 1 second between scans

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Initialize results data array
            results_data = []

            # Iterate through all decoded QR codes
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    print(f"Product found: {product_result[0]}")
                    selected_product = product_result[0]

                    # Call the function to read and insert matched data to pp_out table
                    read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product, results_data)

                else:
                    print("Product not found.")

            # Print the results after processing all decoded QR codes
            for result_row in results_data:
                print(f"Product ID: {result_row[0]}, Count: {result_row[1]}, Excel_count: {result_row[2]}")

            # Wait for 1 second before the next scan
            time.sleep(delay_seconds)

            # Clear the results_data for the next iteration
            results_data.clear()

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__ == "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }

    # Call the QR code scanning function
    scan_qr_code(db_config, excel_file_path)


In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector
import time

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product, results_data):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])

        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])

            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                pcs = row['PCS']

                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)

                # Fetch all rows
                db_results = cursor.fetchall()

                # Process the results as needed
                for db_result in db_results:
                    id, cont = db_result
                    results_data.append([id, cont, pcs])
                print(results_data)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close cursor and connection
        cursor.close()
        conn.close()

def scan_qr_code(db_config):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 2 seconds between scans

    processed_products = set()  # Set to store processed product IDs

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Display the frame with QR codes highlighted
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    product_id = product_result[0]

                    if product_id not in processed_products:  # Check if product not processed yet
                        print(f"Product found: {product_id}")
                        selected_product = product_id

                        # Initialize results data array
                        results_data = []

                        # Call the function to read and insert matched data to pp_out table
                        read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product, results_data)

                        # Print the results
                        for result_row in results_data:
                            print(f"Product ID: {result_row[0]}, Count: {result_row[1]}, Excel_count: {result_row[2]}")

                        # Add the processed product to the set
                        processed_products.add(product_id)

                    # Wait for 2 seconds before the next scan
                    time.sleep(delay_seconds)
                else:
                    print("Product not found.")

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__ == "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }

    # Call the QR code scanning function
    scan_qr_code(db_config)


In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector
import time

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product, results_data):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])

        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])

            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                pcs = row['PCS']

                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)

                # Fetch all rows
                db_results = cursor.fetchall()


                # Process the results as needed
                for db_result in db_results:
                    id, cont = db_result

                    results_data.append([id, cont, pcs])
                print(results_data)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close cursor and connection
        cursor.close()
        conn.close()

def scan_qr_code(db_config):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 2 seconds between scans

    processed_products = set()  # Set to store processed product IDs
    results_data = []  # Initialize results data array outside the loop

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Display the frame with QR codes highlighted
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    product_id = product_result[0]

                    if product_id not in processed_products:  # Check if product not processed yet
                        print(f"Product found: {product_id}")
                        selected_product = product_id

                        # Call the function to read and insert matched data to pp_out table
                        read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product, results_data)

                        # Print the results
                        for result_row in results_data:
                            print(f"Product ID: {result_row[0]}, Count: {result_row[1]}, Excel_count: {result_row[2]}")

                        # Add the processed product to the set
                        processed_products.add(product_id)

                    # Wait for 2 seconds before the next scan
                    time.sleep(delay_seconds)
                else:
                    print("Product not found.")

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__ == "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }

    # Call the QR code scanning function
    scan_qr_code(db_config)


In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector
import time

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product, results_data):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])

        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])

            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                pcs = row['PCS']

                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)
                count=0
                # Fetch all rows
                db_results = cursor.fetchall()


                # Process the results as needed
                for db_result in db_results:
                    id, cont = db_result

                    results_data.append([id, cont, pcs])
                print(results_data)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close cursor and connection
        cursor.close()
        conn.close()

def scan_qr_code(db_config):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 2 seconds between scans

    processed_products = set()  # Set to store processed product IDs
    results_data = []  # Initialize results data array outside the loop

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Display the frame with QR codes highlighted
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    product_id = product_result[0]

                    if product_id not in processed_products:  # Check if product not processed yet
                        print(f"Product found: {product_id}")
                        selected_product = product_id

                        # Call the function to read and insert matched data to pp_out table
                        read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product, results_data)

                        # Print the results
                        for result_row in results_data:
                            print(f"Product ID: {result_row[0]}, Count: {result_row[1]}, Excel_count: {result_row[2]}")

                        # Add the processed product to the set
                        processed_products.add(product_id)

                    # Wait for 2 seconds before the next scan
                    time.sleep(delay_seconds)
                else:
                    print("Product not found.")

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__ == "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }

    # Call the QR code scanning function
    scan_qr_code(db_config)


In [ ]:
def calculate_packed_unpacked(count, excel_count):
    if count == excel_count:
        packed = 1
        unpacked = 0
    elif count > excel_count:
        packed = count / excel_count
        unpacked = 0
    else:
        packed = excel_count / count
        unpacked = 0
    return packed, unpacked


# Take input from the user
count = int(input("Enter count: "))
excel_count = int(input("Enter excel count: "))

packed, unpacked = calculate_packed_unpacked(count, excel_count)
print("Packed:", packed)
print("Unpacked:", unpacked)




In [ ]:
def calculate_packed_unpacked(count, excel_count):
    if count == excel_count:
        packed = 1
        unpacked = 0
    elif count > excel_count:
        packed = count / excel_count
        unpacked = 0
    else:
        packed = 0
        unpacked = 1
    return packed, unpacked



# Take input from the user
count = int(input("Enter count: "))
excel_count = int(input("Enter excel count: "))

packed, unpacked = calculate_packed_unpacked(count, excel_count)
print("Packed:", packed)
print("Unpacked:", unpacked)


In [ ]:
import cv2
from pyzbar.pyzbar import decode
import pandas as pd
import mysql.connector
import time
import math
# Define the calculate_packed_unpacked function
def calculate_packed_unpacked(count, excel_count):
    packed=math.floor(excel_count/count)
    unpacked=excel_count%count
    return packed, unpacked

def read_excel_and_insert_to_pp_out(file_path, db_config, selected_product, results_data):
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(file_path, header=None, skiprows=7, names=['Date', 'SALE ORDER NO.', 'Particulars', 'PCS'])
        df.head(5)
        # Connect to the database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Find the correct column name for 'Particulars'
        excel_particulars_column = 'Particulars'

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Get the product name from the Excel file
            excel_product = str(row[excel_particulars_column])
            #print(excel_product)
            # Check if the Excel product matches the selected product from the QR code
            if excel_product == selected_product:
                pcs = row['PCS']

                # Query the database for the corresponding product
                query = f"SELECT products.id, box.cont FROM products JOIN box ON products.Product = box.item WHERE products.Product = '{excel_product}'"
                cursor.execute(query)
                count=0
                # Fetch all rows
                db_results = cursor.fetchall()

                # Process the results as needed
                for db_result in db_results:
                    id, cont = db_result

                    # Call calculate_packed_unpacked function
                    packed, unpacked = calculate_packed_unpacked(cont, pcs)

                    # Append the results to results_data
                    results_data.append([id, cont, pcs, packed, unpacked])
                    print(f"ID: {id}, Cont: {cont}, PCS: {pcs}, Packable: {packed}, Unpackable(pieces): {unpacked}")
                    print(results_data)
    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close cursor and connection
        cursor.close()
        conn.close()

def scan_qr_code(db_config):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(1)

    # Initialize the database connection and cursor
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cam_on = True  # Flag to control the camera state
    delay_seconds = 1  # Delay for 2 seconds between scans

    processed_products = set()  # Set to store processed product IDs
    results_data = []  # Initialize results data array outside the loop

    while cam_on:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if ret:
            # Decode QR codes from the frame
            decoded_objects = decode(frame)

            # Display the frame with QR codes highlighted
            for obj in decoded_objects:
                qr_data = obj.data.decode('utf-8')
                print("Decoded QR Code:", qr_data)

                # Check if the first 23 characters of the decoded QR code are present in the pro_code column in the products table
                pro_code_query = f"SELECT Product FROM products WHERE pro_code = '{qr_data[:23]}'"
                cursor.execute(pro_code_query)
                product_result = cursor.fetchone()

                if product_result:
                    product_id = product_result[0]

                    if product_id not in processed_products:  # Check if product not processed yet
                        print(f"Product found: {product_id}")
                        selected_product = product_id

                        # Call the function to read and insert matched data to pp_out table
                        read_excel_and_insert_to_pp_out(excel_file_path, db_config, selected_product, results_data)

                        # Add the processed product to the set
                        processed_products.add(product_id)

                    # Wait for 2 seconds before the next scan
                    time.sleep(delay_seconds)
                else:
                    print("Product not found.")

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

        # Check if 'q' key is pressed to turn off the camera
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Add a short delay to ensure the window is closed before moving on

    # Close the database connection
    cursor.close()
    conn.close()

if __name__ == "__main__":
    # Get the Excel file path from the user
    excel_file_path = input("Enter the path of the Excel file: ")

    # Define the database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'database': 'paccapro',
    }


    # Call the QR code scanning function
    scan_qr_code(db_config)


In [ ]:
def calculate_packed_unpacked(count, excel_count):
    if count == excel_count:
        packed = 1
        unpacked = 0
    elif count > excel_count:
        packed = count / excel_count
        unpacked = 0
    else:
        packed = 0
        unpacked = 1
    return packed, unpacked

# Take input from the user
count = int(input("Enter count: "))
excel_count = int(input("Enter excel count: "))

packed, unpacked = calculate_packed_unpacked(count, excel_count)
print("Packed:", packed)
print("Unpacked:", unpacked)
